In [ ]:
import warnings

import cirq
import numpy as np
import matplotlib.pyplot as plt
import bloqade.cirq_utils as utils
from cirq.contrib.svg import SVGCircuit

from bloqade import squin, cirq_utils

warnings.filterwarnings("ignore")

def build_state_injection(theta: float, phi: float) -> cirq.Circuit:
    """
    Build state injection that attaches 3 stabalizers to 1 input state
    Inputs:
    theta in [0,pi), phi in [0, 2pi)
    Returns:
    Circuit
    """

    @squin.kernel
    def state_injection_kernal():
        """
        Encode data qbit
        """
        q_data = squin.qalloc(7)
        squin.u3(theta, phi, 0, q_data[6])

        # apply sqrt_y_adj to first 5 gates
        for i in range(6):
            squin.sqrt_y_adj(q_data[i])

        # apply cz to pairs
        squin.cz(q_data[1], q_data[2])
        squin.cz(q_data[3], q_data[4])
        squin.cz(q_data[5], q_data[6])

        # apply sqrt_y gate to injection state
        squin.sqrt_y(q_data[6])

        # apply cz to more pairs
        squin.cz(q_data[0], q_data[3])
        squin.cz(q_data[2], q_data[5])
        squin.cz(q_data[4], q_data[6])

        # apply sqrt_y gates
        for i in range(2, 7):
            squin.sqrt_y(q_data[i])

        # apply MORE cz gates
        squin.cz(q_data[0], q_data[1])
        squin.cz(q_data[2], q_data[3])
        squin.cz(q_data[4], q_data[5])

        squin.sqrt_y(q_data[1])
        squin.sqrt_y(q_data[2])
        squin.sqrt_y(q_data[4])

        """
        Encode auxillatry qbit
        """
        q_aux = squin.qalloc(7)
        squin.u3(np.pi/2, 0, 0, q_aux[6])

        # apply sqrt_y_adj to first 5 gates
        for i in range(6):
            squin.sqrt_y_adj(q_aux[i])

        # apply cz to pairs
        squin.cz(q_aux[1], q_aux[2])
        squin.cz(q_aux[3], q_aux[4])
        squin.cz(q_aux[5], q_aux[6])

        # apply sqrt_y gate to injection state
        squin.sqrt_y(q_aux[6])

        # apply cz to more pairs
        squin.cz(q_aux[0], q_aux[3])
        squin.cz(q_aux[2], q_aux[5])
        squin.cz(q_aux[4], q_aux[6])

        # apply sqrt_y gates
        for i in range(2, 7):
            squin.sqrt_y(q_aux[i])

        # apply MORE cz gates
        squin.cz(q_aux[0], q_aux[1])
        squin.cz(q_aux[2], q_aux[3])
        squin.cz(q_aux[4], q_aux[5])

        squin.sqrt_y(q_aux[1])
        squin.sqrt_y(q_aux[2])
        squin.sqrt_y(q_aux[4])

        """
        Stean circuit
        """

    # Create LineQubits for compatibility with existing code
    qubits = cirq.LineQubit.range(14)
    circuit = cirq_utils.emit_circuit(state_injection_kernal, circuit_qubits=qubits)

    return circuit

state_injection_circuit = build_state_injection(np.pi,np.pi)
SVGCircuit(state_injection_circuit)

IndexError: list index out of range